In [ ]:
# downloading dataset
import pandas as pd
money = pd.read_csv('M2SLMoneyStock.csv')
spending = pd.read_csv('PCEPersonalSpending.csv')
df =pd.merge(money, spending, on='Date', how='outer')
#df2 = pd.merge(passengers, restaurant, on='Date', how='outer')
# set index to Date
df = df.set_index('Date')
# convert index into datetime format
df.index = pd.DatetimeIndex(df.index).to_period('m')
df

In [ ]:
#checking stationarity, 
# Cointegration is a statistical method used to test the correlation between two or more non-stationary time series 
# in the long run or for a specified period
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues

#coint_johansen(endog, det_order, k_ar_diff); k_ar_diff Number of lagged differences in the model.
# det_order: determintation order; -1 no-determinstic terms: the order of time polynomial in the null-hypothesis
data= df.dropna()
# Eigenvalues of  coefficient matrix
result=coint_johansen(data,-1,1).eig
print(result)

In [ ]:
df

In [ ]:
# splitting the data into train and validation sets: 90% for training, 10% for validation
data= df.dropna()
train = data[:int(0.9*(len(df)))]
test = data[int(0.9*(len(df))):]

In [ ]:
# using vector autogregression model 
from statsmodels.tsa.vector_ar.var_model import VAR
# a pth-order VAR refers to a VAR model which includes lags for the last p time periods
# fit(maxlags): Maximum number of lags to check for order selection,
# defaults to 12 * (nobs/100.)**(1./4)
# we set order = 5 
model = VAR(endog=train, freq='M')
fitted_var = model.fit(5)# fitted_var.summary(): to access model output


# make predictions and evaluate the model on validation set
# VARResults.forecast: Produce linear minimum MSE forecasts for desired number of steps ahead, using prior values y
lagged_Vals = train.values[-10:]# The VAR .forecast() requires passing in a lag order number of previous observations.
predictions = fitted_var.forecast(y=lagged_Vals, steps=len(test))

In [ ]:
#the predictions are made as an array, where each list represents the predictions of a row in the dataset.
# we need first to convert the array into  a dataframe
from sklearn.metrics import mean_absolute_percentage_error
# locate the predictions in a new dataframe with the same col names: without 'CO(GT)'
cols =df.columns
idx = test.index
pred = pd.DataFrame(index=idx,columns=[cols])
for j in range(0,2):
    for i in range(0, len(predictions)):
        pred.iloc[i][j] = predictions[i][j]

#calculate MAPE for all cols
for col in cols:
    print('MAPE value for', col, 'is : ', mean_absolute_percentage_error(pred[col], test[col]))

In [ ]:
#plot cols against predictions
import matplotlib.pyplot as plt
fig = plt.subplots(figsize=(8,5))
test2 = test.to_timestamp(freq='d')

df_forecast=pd.DataFrame(data=pred, index=idx, columns=cols)
plt.plot(test2['Money'],color="green", label ='Money')
plt.plot(pred['Money'], color="blue", label ='Money forecasts')
plt.plot(test2['Spending'],color="red", label ='Spending')
plt.plot(pred['Spending'], color="black", label ='Spending forecasts')
plt.legend(loc="upper left")
plt.show()
# since each variable is a linear function of past lags of itself and past lags of the other variables, we get fitted lines

In [ ]:
# using LSTM;
# first we need to scale input and output of LSTM using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled =scaler.fit_transform(train)
x_train_scaled.shape 

In [ ]:
#using money variable as input time series
scaler2= StandardScaler()
y_train_scaled =scaler2.fit_transform(train[['Money']])
y_train_scaled.shape

In [ ]:
# for lag= 5 we need to create the sequences of the input array manually;reshaping input data
import numpy as np
x_train=[]
y_train = []
for i in range(5, 226):
    x_train.append(x_train_scaled[i-5:i])
    y_train.append(y_train_scaled[i][0])
x_train= np.array(x_train)    
y_train = np.array(y_train) 
print(x_train.shape)

In [ ]:
# build LSTM model 
from tensorflow import keras
from tensorflow.keras.layers import  Dropout, Dense, LSTM
from tensorflow.keras.models import Sequential

model = keras.Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(5,2)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam')
history = model.fit(x_train, y_train, epochs=200, batch_size= 32, shuffle=False, validation_split=0.2)

In [ ]:
# appending the last 5 of train set to the test set: to be used for forecasting, since we use 5 lags 
train_last_5= train[-5:]
extended_test= pd.concat((train_last_5, test), axis=0)
# rescaling test data
scaled_test= scaler.fit_transform(extended_test)
scaled_test.shape

# rescaling y 
y_test_scaled = scaler2.fit_transform(extended_test[['Money']])
y_test_scaled.shape

In [ ]:
# reshaping input data for testing the model
x_test=[]
for i in range(5, 31):
    x_test.append(scaled_test[i-5:i])    
x_test= np.array(x_test)    
print(x_test.shape)

In [ ]:
# forecasting on test data
y_test=model.predict(x_test)
# inverse scaling of money forecasts
y= scaler2.inverse_transform(y_test)
y

In [ ]:
# plotting predictions against actual money values
fig = plt.subplots(figsize=(10,6))
test2 = test.to_timestamp(freq='d')
df_forecast=pd.DataFrame(data=y, index=idx)
plt.plot(test2['Money'],color="red", label ='Money')
plt.plot(df_forecast, color="blue", label ='Money forecasts')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# MAPE between forecasts and actual money
print('MAPE value for Money is:', mean_absolute_percentage_error(y, test['Money']))
